In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy import stats
import yfinance as yf

In [59]:
data = pd.read_excel('stocks_info.xlsx')
data = data.iloc[:1210]
data

,代號,名稱,年月,開盤價(元)_年,最高價(元)_年,最低價(元)_年,收盤價(元)_年,成交量(百萬股)_年,成交值(百萬元)_年,報酬率％_年,...,本益比-TEJ,股價淨值比-TSE,股價淨值比-TEJ,股價營收比-TEJ,股利殖利率-TSE,現金股利率,股價漲跌(元),高低價差%,市場別,該年結束日
0,0050,元大台灣50,2023/12,103.24,132.90,102.15,132.36,2434,303205,27.3980,...,NaN,NaN,NaN,NaN,NaN,3.03,25.25,25.0000,TSE,2023/12/29
1,0051,元大中型100,2023/12,49.45,73.95,49.01,73.65,16,1086,48.9435,...,NaN,NaN,NaN,NaN,NaN,3.39,22.40,46.4390,TSE,2023/12/29
2,0052,富邦科技,2023/12,84.99,124.56,84.44,124.22,77,8726,45.4004,...,NaN,NaN,NaN,NaN,NaN,3.09,36.90,41.4595,TSE,2023/12/29
3,0053,元大電子,2023/12,50.29,72.15,50.00,71.65,2,146,43.2803,...,NaN,NaN,NaN,NaN,NaN,2.65,20.25,40.3696,TSE,2023/12/29
4,0055,元大MSCI金融,2023/12,20.92,24.27,20.68,24.12,50,1160,15.0476,...,NaN,NaN,NaN,NaN,NaN,2.49,2.62,15.4884,TSE,2023/12/29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1205,9943,好樂迪,2023/12,59.54,108.87,59.25,90.50,130,11928,50.9421,...,13.8986,2.88,2.89,4.02,3.65,3.65,28.30,82.7974,TSE,2023/12/29
1206,9944,新麗,2023/12,19.47,23.29,19.05,20.20,56,1213,3.5240,...,NaN,0.60,0.62,1.00,5.20,5.20,-0.25,26.1614,TSE,2023/12/29
1207,9945,潤泰新,2023/12,46.62,49.26,32.35,37.75,1356,50023,-19.0663,...,14.0512,1.26,1.26,3.68,0.00,0.00,-5.50,30.7514,TSE,2023/12/29
1208,9946,三發地產,2023/12,11.83,21.65,11.39,20.40,68,1130,72.4134,...,121.4177,1.04,1.04,7.77,2.11,2.10,8.20,81.1475,TSE,2023/12/29


In [60]:
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

rows_list = []
for index, row in data.iterrows():
    symbol = row['代號']  # 访问股票代碼列
    name = row['名稱']    # 访问股票名稱列
    year1ChangePercent = row['報酬率％_年']
    
    new_list = [
        symbol,
        name,
        year1ChangePercent,
        'N/A'
    ]
    rows_list.append(new_list)

momentum = pd.DataFrame(rows_list, columns=my_columns)
momentum.head()

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,0050,元大台灣50,27.3980,N/A
1,0051,元大中型100,48.9435,N/A
2,0052,富邦科技,45.4004,N/A
3,0053,元大電子,43.2803,N/A
4,0055,元大MSCI金融,15.0476,N/A


In [64]:
momentum = momentum.sort_values(by='One-Year Price Return', ascending=False)